# ☕ Data Extraction: Merida Coffee Shops

This project begins with the **data acquisition phase**, using the **Google Places API** via the **Google Maps Platform**.  
This API is selected for its comprehensive and regularly updated geospatial and business information, suitable for performing location-based analyses.

The objective of this phase is to extract the location and metadata for all coffee shops located within the municipal boundaries of **Merida, Yucatan**.

***

## Google Places API Service Selection

The **Places Text Search (New)** service is used because it allows flexible querying based on a text string and returns a list of matching places along with relevant details such as coordinates, ratings, and price levels.

## 1. Defining the Search Area: Location Restriction

The Google Places API provides two parameters to limit search results geographically: `locationBias` and `locationRestriction`.  
For this project, the focus is on enforcing strict adherence to the Merida municipal boundary.

| Method | Description | Notes |
| :--- | :--- | :--- |
| **`locationBias`** | Specifies a preferred region for search results, but results outside this region may still be included. | Not suitable for analyses requiring strict geographic boundaries. |
| **`locationRestriction`** | Defines a bounding box (rectangle viewport) from which all results must originate. Results outside this area are excluded. | Selected option: ensures all data points are within the defined Merida boundary, maintaining geographic precision and avoiding overlaps. |

The search area is defined using a **rectangle viewport** with the `locationRestriction` parameter to ensure a consistent and boundary-aligned dataset.

### Gridding the City with Folium

To ensure complete coverage of Merida and avoid omissions due to the **Google Places API’s 20-result limit per query**, the city’s bounding box is divided into smaller **rectangular viewports** (a process known as *gridding*).

Each viewport defines a separate `locationRestriction` (SW and NE coordinates) and is queried independently.  
This approach minimizes data loss in high-density areas and ensures comprehensive data collection.

#### Grid Generation and Verification

The **Folium** library is used to visualize and verify the grid layout. Each generated rectangle corresponds to a specific API query area, ensuring full geographic coverage and facilitating validation of the extraction process.

In [2]:
import folium

#Create the map of Merida using an approximate center point.
m = folium.Map(location=[20.9939879883004, -89.62853393602846], zoom_start=12)
delta = 0.0135 #The variable “delta” controls how “large” each rectangle is.

initial_lat, initial_lng = 20.891532412575916, -89.73272017481521 #This is the initial SW point where the loop start to create the other viewport rectangles. 


#The points generated will be stored and will be used when the API is called.
rectangles_viewports = []

for i in range(15):
    for j in range(15):
        low = [initial_lat+i*delta, initial_lng+j*delta]
        high = [initial_lat+(i+1)*delta, initial_lng+(j+1)*delta]
        folium.Rectangle(
            bounds = [low, high],
            tooltip = f'({i+1},{j+1})',
            fill = True
        ).add_to(m)
        
        rectangles_viewports.append((tuple(low), tuple(high)))
m

### 2. Data Acquisition: Iterative Grid Search
This section describes the bulk data extraction process using the grid of bounding box coordinates generated in the previous step. An individual API request is performed for each of the 225 grid cells to ensure complete spatial coverage of the Merida municipality.